In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits
import pymcfost as mcfost
mcfost.__version__
#%matplotlib inline

#p_dirin -> directory with mcfost files .para e RT.fits.gz
#sim_para -> dict with parameters that need to be added to the header
#function that generates the post pymcfost image with added information in the header
def generateImage(p_dirin,
                  p_bmaj,
                  p_bmin,
                  p_bpa = 0,
                  p_dirout='./',
                  fileout='ris2.fits',
                  p_Jy=True,
                  p_perbeam=True,
                  p_i=0,
                  sim_para={},
                  ax=None
                 ):
    
    image = mcfost.Image(p_dirin)
    pltimage = image.plot(i=p_i, bpa=p_bpa, bmaj=p_bmaj,
                          bmin=p_bmin,scale='lin',Jy=p_Jy, per_beam=p_perbeam,
                          no_ylabel=True,no_xlabel=True,
                          plot_stars=False,limits=[0.8,-0.8,-0.8,0.8], ax=ax, title=fileout)
    header = image.header
    
    #manually updating header.BUNIT
    bunit_1 = 'W.m-2'
    bunit_2 = '.pixel-1'
    if p_Jy:
        bunit_1 = 'Jy'
    if p_perbeam:
        bunit_2 = '.beam-1'
    header['BUNIT'] = bunit_1 + bunit_2
    
    #adding simulation parameters to the header
    
    #star properties
    nstars = image.P.simu.n_stars
    header['NSTARS'] = nstars
    for i in range(0, nstars):
        header['TSTAR' + str(i)] = image.P.stars[i].Teff, 'K'
        header['MSTAR' + str(i)] = image.P.stars[i].M, 'M_sun'
        header['RSTAR' + str(i)] = image.P.stars[i].R, 'R_sun'
        
    #beam informations
    header['BMAJ'] = p_bmaj
    header['BMIN'] = p_bmin
    header['BPA'] = p_bpa, 'deg'
    
    #adding info in sim_para dict
    for key in sim_para:
        header[key] = sim_para[key]
        
    #inclination and position angle
    header['DISKPA'] = image.P.map.PA, 'deg'
    header['IMIN'] = image.P.map.RT_imin, 'deg'
    header['IMAX'] = image.P.map.RT_imax, 'deg'
    header['NI'] = image.P.map.RT_ntheta
    header['IUSED'] = p_i
    
    #distance
    header['DISTANCE'] = image.P.map.distance, 'pc'
    
    #saving fits file
    image.writeto(filename=p_dirout+'/' + fileout)
    

In [2]:
#function that extracts informations from image.log file
def get_imagelog_info(sim_para, p_dir):
    logf = open(p_dir+'/image.log')
    data = logf.read()
    
    #read gass mass and dust mass
    i_mg = data.find('gas mass')
    str_mg = data[i_mg:i_mg+data[i_mg:].find('\n')].split(' ')
    sim_para['MGAS']  = (float(str_mg[-2]), str_mg[-1])
    if str_mg[-1]!='Msun' :
        print('Error: gas mass not in M_sun units')
    i_md = data.find('dust mass')
    str_md = data[i_md:i_md+data[i_md:].find('\n')].split(' ')
    sim_para['MDUST']  = (float(str_md[-2]), str_md[-1])
    if str_md[-1]!='Msun' :
        print('Error: dust mass not in M_sun units')
        
    #read planet mass and orbital radius
    i_sink = []
    i_sink.append(data.find('Sink'))
    i_sink.append(i_sink[-1] + data[i_sink[-1]+1:].find('Sink'))
    while i_sink[-1]-i_sink[-2]!= 0:
        i_sink.append(i_sink[-1] + 1+ data[i_sink[-1]+1:].find('Sink'))
    i_sink.pop(-1)
    i_sink.pop(0)
    i_sink.append(i_sink[-1]+200)
    str_sink = [data[i_sink[i]:i_sink[i+1]] for i in range(0, len(i_sink)-1)]

    sink_index = 0
    for info in str_sink:
        if info.find('distance') > 0:
            info = " ".join(info.split())
            i_m = info.find('M')
            str_m = info[i_m:i_m+1+info[i_m+1:].find('M')+2]
            sim_para['MPLANET'+str(sink_index)] = float(str_m.split('=')[-1].split(' ')[-2]), str_m.split('=')[-1].split(' ')[-1]
            i_r = info.find('distance')
            str_r = info[i_r:i_r+1+info[i_r+1:].find('au')+2]
            sim_para['RORBP'+str(sink_index)] = float(str_r.split('=')[-1].split(' ')[-2]), str_r.split('=')[-1].split(' ')[-1]
            sink_index+=1
    sim_para['NPLANETS'] = sink_index

In [31]:
import os
import re
import hashlib
from pathlib import Path

def read_dirs_doimages(parent_dir='./', beam_dims=[0.1]):
    
    #setting global parameters, values for  dstau simulations
    sim_para = 
    {
        'RIN': (10, 'au, initial disk inner radius' ),
        'ROUT': (100, 'au, initial disk outer radius'),
        'RC': (70, 'au, radius of exponential taper'),
        'HRIN': (0.6,'aspect ratio at R_in'),
        'ALPHASS': (0.005, 'Shakura-Sunyaev viscosity'),
        'RHOG': (1, 'g/cm^3, intrinsic grain density'),
        'PINDEX': (1, 'power law index for gas surface density profile'),
        'QINDEX': (0.25, 'power law index for sound speed profile'),
    }
    
    current_dir = parent_dir
    key = 0
    out_dir = parent_dir + 'processed_fits/'
    out_dir_content = ''
    
    #setting up plotting
    key_plots = 0
    curr_sub = 0
    #please don't set rows or cols to 1!
    plot_rows = 2
    plot_cols = 2
    fig, axs = plt.subplots(plot_rows, plot_cols)
    fig.set_size_inches(5*plot_cols,5*plot_rows)
    
    #getting dir content
    dir_cont = os.listdir(parent_dir)
    
    #creating dir for the fits file that will be produced
    if 'processed_fits' not in dir_cont:
        os.mkdir(parent_dir + 'processed_fits')
        out_dir = parent_dir + 'processed_fits/'
        Path(out_dir+'source_hashes.sha1').touch()
    else:
        #if the output dir already exists check for last key and update
        #retrieving output dir content 
        out_dir_content = os.listdir(out_dir)
        
        #if .fits exists get max key already present and restart from there
        fits_sim_mod = re.compile("^\d{6}MP\d+_time\d+gd\d+W\d+B\d+.fits$")
        keys = [int(filename[:6]) for filename in out_dir_content if fits_sim_mod.match(filename)]
        if keys != []:
            key = max(keys) + 1
        
        #if _images.png exists get max key_plots already present and restart from there
        im_mod = re.compile("^\d{3}_images.png$")
        keys = [int(filename[:3]) for filename in out_dir_content if im_mod.match(filename)]
        if keys != []:
            key_plots = max(keys) + 1
            
        if 'source_hashes.sha1' not in out_dir_content:
            Path(out_dir+'source_hashes.sha1').touch()
            
    #cleaning dir content
    dir_mod = re.compile("^MP[0-9]+_time[0-9]+$")
    subdirs1 = [sd for sd in dir_cont if dir_mod.match(sd)]
    
    #cicle for /MP#_time#/ 
    for sub_dir1 in subdirs1:
        
        #getting time of simulation
        sim_para['TIME'] = int(sub_dir1.split('time')[-1]), 'planet orbits'
        
        #getting subdirs with the correct format
        dir_cont2 = os.listdir(parent_dir+sub_dir1)
        dir_mod2 = re.compile("^gd[0-9]+$")
        subdirs2 = [sd for sd in dir_cont2 if dir_mod2.match(sd)]
        
        #cicle for /MP#_time#/gd#/
        for sub_dir2 in subdirs2:
            
            #from now on using sim_para_upd to avoid
            #having to delete info that are not overrided
            sim_para_upd = sim_para 
            
            #this directory contains image.log and data_### dirs
            #updating sim_para with info contained in image.log
            get_imagelog_info(sim_para=sim_para_upd, p_dir=parent_dir+sub_dir1+'/'+sub_dir2)
            
            #searching for data_### dirs
            #getting subdirs
            dir_cont3 = os.listdir(parent_dir+sub_dir1+'/'+sub_dir2)
            dir_mod3 = re.compile("^data_[0-9]+$")
            subdirs3 = [sd for sd in dir_cont3 if dir_mod3.match(sd)]
            
            #cicle for /MP#_time#/gd#/data_#
            for sub_dir3 in subdirs3:
                
                current_dir = parent_dir + sub_dir1 + '/' + sub_dir2 + '/' + sub_dir3 + '/'
                
                for beam_dim in beam_dims:
                    
                    #select inclination
                    #TODO: handle multiple inclinations
                    p_i = 0
                    
                    #check if a new image for plots needs to be created
                    if curr_sub == plot_rows*plot_cols:
                            #save plot
                            plt.tight_layout()
                            print('generating image ' + str(key_plots) + ' ...', end='')
                            plt.savefig(out_dir + str(key_plots).zfill(3) + '_images.png')
                            plt.close()
                            print('done')
                            
                            #generating new image
                            curr_sub = 0
                            fig, axs = plt.subplots(plot_rows, plot_cols)
                            fig.set_size_inches(10*plot_cols,10*plot_rows)
                            key_plots+=1
                    
                    #generate file name
                    key_str = str(key).zfill(6)
                    wave_str = sub_dir3.split('data_')[-1]
                    out_filename = key_str + sub_dir1 + sub_dir2 + 'W' + wave_str + 'B' + str(beam_dim).split('.')[-1] + '.fits'
                    print(key_str + ' - generating ' + out_filename[6:] + '  \t...', end='')
                    info_to_hash = 'bd' + str(beam_dim) +'ii'+ str(p_i) + 'fn' + out_filename[6:]
                    
                    #check if RT.fits.gz has alredy been processed
                    hashes_f = open(out_dir + 'source_hashes.sha1')
                    hashes = [line.split('-')[-1].strip() for line in hashes_f.read().split('\n')]
                    RTfile = open(current_dir + 'RT.fits.gz', 'rb')
                    xHash = hashlib.sha1((hashlib.sha1(RTfile.read()).hexdigest()+info_to_hash).encode()).hexdigest()
                    RTfile.close() 
                    hashes_f.close()

                    if xHash not in hashes:
                        generateImage(current_dir, p_bmaj=beam_dim, 
                                      p_bmin=beam_dim, p_dirout=out_dir, 
                                      fileout=out_filename, sim_para=sim_para_upd, ax=axs[int(curr_sub/plot_cols), curr_sub%plot_cols])
                        key+=1
                        hashes_f = open(out_dir + 'source_hashes.sha1', 'a')
                        hashes_f.write(key_str + ' - ' + xHash + '\n')
                        hashes_f.close()
                        
                        print('done')
                        curr_sub+=1
                        
                    else:
                        print('exists, skipped')
                        
    print('generating image ' + str(key_plots) + ' ...', end='')
    plt.tight_layout()
    plt.savefig(out_dir + str(key_plots).zfill(3) + '_images.png') 
    print('done')
    plt.close()

In [10]:
from astropy import constants as const

def HR_profile(r, hrin=0.06, rin=10, q=0.25):
    return hrin*(r/rin)**(0.5-q)

def cs_profile(r, hrin=0.06, rin=10, q=0.25, M_st=0.83):
    v_k  = np.sqrt(const.GM_sun*M_st/r)
    return HR_profile(r=r, hrin=hrin, rin=rin, q=q)

def T_profile(r, hrin=0.06, rin=10, q=0.25, M_st=0.83, rho_g=1, s=1 ):
    return rho_g*4*np.pi*(s*(10**6))**3/(3*const.k_B*1000)*(cs_profile(r, hrin=hrin, rin=rin, q=q, M_st=M_st)**2)

In [11]:
def computable_features(sim_para):
    
    #computing features
    s = sim_para['WAVE']/(2*np.pi)
    rp = sim_para['RORBP']
    hrp = HR_profile(rp, hrin=sim_para['HRIN'], rin=sim_para['RIN'], q=sim_para['QINDEX'])
    Trp = T_profile(rp, hrin=sim_para['HRIN'], rin=sim_para['RIN'], q=sim_para['QINDEX'], M_st=sim_para['MSTAR0'], rho_g=sim_para['RHOG'], s=s)
    
    #adding them to sim_para
    sim_para['GRSIZE'] = (s, 'grain size')
    sim_para['HR'] = (hrp, 'aspect ratio at orbital radius')
    sim_para['TEMP'] = (Trp, 'temperature at orbital radius')

In [32]:
parent_dir='../simulazioni/'
read_dirs_doimages(parent_dir=parent_dir)

000000 - generating MP2_time100gd120W1300B1.fits  	...done
000001 - generating MP2_time100gd150W1300B1.fits  	...done
000002 - generating MP1_time120gd10W1300B1.fits  	...done
000003 - generating MP1_time120gd120W1300B1.fits  	...done
generating image 0 ...done
000004 - generating MP1_time050gd10W1300B1.fits  	...done
000005 - generating MP1_time050gd110W1300B1.fits  	...done
generating image 1 ...done


TODOs:

    - study possible values for beam_size
    - update doc
    - add info on T and opacity
    
    - improve get_imagelog_info() with the use of re
    
    -change read_imagelog to  actually update sim_para

In [223]:
filegen = fits.open('fit7.fits')
filegen.info()

Filename: fit7.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      40   (1024, 1024)   float64   


In [224]:
filegen[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1024                                                  
NAXIS2  =                 1024                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
WAVE    =                1300. / wavelength [microns]                           
CTYPE1  = 'RA---TAN'                                                            
CRVAL1  =                   0. / RAD                                            
CRPIX1  =                  513                                                  
CDELT1  =        -5.118252E-07 / pixel scale x [deg]                            
CUNIT1  = 'deg     '        

In [273]:
import re
prova = ['MP001_time66', 'MP001_time664', 'altro', 'MP001_time661']
dir_mod = re.compile("MP[0-9]+_time[0-9]+")
provafilt = [str for str in prova if dir_mod.match(str)]
provafilt

['MP001_time66', 'MP001_time664', 'MP001_time661']

In [264]:
print(dom)

<re.Match object; span=(0, 12), match='MP001_time66'>


In [283]:
str1

'fkf'

In [3]:
datat = 'dat1300'
datat.split('dat')[-1]

'1300'

In [32]:
stringa = '12345ciao'
stringa[5:]

'ciao'

In [29]:
sprove = 'data_1300'
dir_mod3 = re.compile("^data_[0-9]+$")
print(dir_mod3.match(sprove))

<re.Match object; span=(0, 9), match='data_1300'>


In [3]:
import hashlib

file = open('fitsFile/data_1300/RT.fits', 'rb')
hash_object = hashlib.sha1(file.read())
pbHash = hash_object.hexdigest()
file.close()

pbHash

'77e6c1419a030a46c9726e69235e5fe1dc96086d'

In [49]:
prova3 = 'stringaaaaa'
bytes(prova3.encode())

b'stringaaaaa'

In [76]:
keyy = '123456mckc'
keyy[0:6]

'123456'

In [8]:
def change(a):
    a=10
a= 9
print(a)
change(a=a)
print(a)

9
9
